# Day 12: GUI Development - Building Web Interfaces

## From CLI to Beautiful Web UIs

You've mastered agents - now make them accessible to everyone with WebUI! Today you'll learn how to create production-ready web interfaces.

### Why GUI?

**CLI**: Great for developers
```python
for response in bot.run(messages):
    print(response)  # Only developers can use this
```

**WebUI**: Accessible to everyone
```python
WebUI(bot).run()  # Anyone with a browser can use this!
```

### What is WebUI?

**WebUI** is Qwen-Agent's built-in Gradio-based interface:
- 🎨 Beautiful chat interface
- 📁 File upload support
- 💬 Conversation history
- 🔄 Real-time streaming
- 🚀 Production-ready

### Today's Journey:
1. **Basic WebUI** - From assistant_qwen3.py
2. **chatbot_config** - Customization options
3. **File uploads** - RAG in the browser
4. **Tools in WebUI** - CodeInterpreter + MCP
5. **Advanced Gradio** - From group_chat_demo.py
6. **Deployment** - Share, Docker, production

Let's build beautiful UIs! 🎨

---
## Part 1: Setup

Same Fireworks API configuration.

In [ ]:
import os
import json

os.environ['FIREWORKS_API_KEY'] = 'fw_3ZTLPrnEtuscTUPYy3sYx3ag'

llm_cfg = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {'max_tokens': 32768, 'temperature': 0.6}
}

print('✅ Fireworks API configured')

---
## Part 2: Basic WebUI - Your First Web Interface

### From Official assistant_qwen3.py

The simplest WebUI is just one line!

In [ ]:
from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI

# Create a basic assistant
bot = Assistant(
    llm=llm_cfg,
    name='My First WebUI Bot',
    description='A friendly assistant with a web interface'
)

print("✅ Created assistant for WebUI")
print("\nTo launch the web interface, you would run:")
print("WebUI(bot).run()")
print("\nThis opens a browser with:")
print("- Chat interface")
print("- Message history")
print("- File upload button")
print("- Copy button for responses")

### What WebUI Does Automatically:

1. **Creates Gradio Interface**
   - Chat window with message history
   - Input box for typing messages
   - File upload button

2. **Manages State**
   - Conversation history
   - Uploaded files
   - Agent responses

3. **Handles Streaming**
   - Real-time response display
   - Token-by-token streaming
   - Smooth user experience

4. **File Support**
   - Upload documents
   - Automatic RAG
   - Multiple file formats

---
## Part 3: chatbot_config - Customization Options

### From assistant_qwen3.py (Line 117-122)

Customize the chat interface with `chatbot_config`:

In [ ]:
# From official assistant_qwen3.py - chatbot_config example
chatbot_config = {
    'prompt.suggestions': [
        'What time is it?',
        'https://github.com/orgs/QwenLM/repositories Extract markdown content of this page, then draw a bar chart to display the number of stars.'
    ]
}

print("Chatbot Config Options:\n")
print("1. prompt.suggestions - Suggested prompts shown to users")
print("   Example: 'What time is it?'")
print("   Can include complex tasks with URLs\n")

print("Usage:")
print("WebUI(bot, chatbot_config=chatbot_config).run()")

### Common chatbot_config Options:

```python
chatbot_config = {
    # Suggested prompts (shown as clickable buttons)
    'prompt.suggestions': [
        'Tell me a joke',
        'What can you do?',
        'Analyze this document'
    ],
    
    # More options can be customized in advanced usage
}
```

**Why suggestions?**
- Help users get started
- Show agent capabilities
- Reduce friction for new users

---
## Part 4: Assistant with Tools in WebUI

### From assistant_qwen3.py (Line 64-82)

WebUI works perfectly with tools!

In [ ]:
# From official assistant_qwen3.py - tools configuration
tools = [
    {
        'mcpServers': {  # MCP tool servers
            'time': {
                'command': 'uvx',
                'args': ['mcp-server-time', '--local-timezone=Asia/Shanghai']
            },
            'fetch': {
                'command': 'uvx',
                'args': ['mcp-server-fetch']
            }
        }
    },
    'code_interpreter',  # Built-in tool
]

print("Tools Configuration (from official example):\n")
print("✅ MCP Tools:")
print("   - time: Get current time")
print("   - fetch: Fetch web content\n")
print("✅ Built-in Tools:")
print("   - code_interpreter: Execute Python code\n")

print("Note: MCP tools require Node.js/uvx installed")
print("For Jupyter, we'll use just code_interpreter")

In [ ]:
# Create assistant with code_interpreter for WebUI
tool_bot = Assistant(
    llm=llm_cfg,
    function_list=['code_interpreter'],
    name='Coding Assistant',
    description='I can write and execute Python code for you!'
)

# Test in CLI (same bot works in WebUI)
messages = [{'role': 'user', 'content': 'Calculate 12345 * 67890'}]

print("Testing tool bot (CLI mode):\n")
for response in tool_bot.run(messages):
    for msg in response:
        if msg.get('function_call'):
            print(f"🔧 Using: {msg['function_call']['name']}")
        elif msg.get('content'):
            print(f"Result: {msg['content'][:150]}\n")
            break

print("\n✅ This same bot works in WebUI:")
print("WebUI(tool_bot).run()")
print("Users can ask it to calculate, plot, analyze data!")

---
## Part 5: RAG with File Uploads

### Enable Document Upload in WebUI

WebUI automatically provides file upload when you use the `files` parameter!

In [ ]:
# Create a sample document
sample_doc = """Product Manual

Chapter 1: Getting Started
To start the device, press the power button for 3 seconds.
The LED will turn green when ready.

Chapter 2: Troubleshooting
If the device doesn't start:
1. Check the battery level
2. Ensure the power adapter is connected
3. Hold reset button for 10 seconds

Chapter 3: Maintenance
Clean the device monthly with a soft cloth.
Avoid water and direct sunlight.
"""

with open('product_manual.txt', 'w') as f:
    f.write(sample_doc)

print("✅ Created sample document\n")

In [ ]:
# Method 1: Pre-load files
rag_bot_preloaded = Assistant(
    llm=llm_cfg,
    name='Product Support',
    description='I help with product questions',
    files=[os.path.abspath('product_manual.txt')]
)

print("Method 1: Pre-loaded Files")
print("- Files are loaded when bot is created")
print("- Users can't upload new files")
print("- Good for fixed documentation\n")

# Test
messages = [{'role': 'user', 'content': 'How do I start the device?'}]
for response in rag_bot_preloaded.run(messages):
    if response:
        print(f"Answer: {response[-1].get('content', '')[:150]}...\n")
        break

In [ ]:
# Method 2: Allow user uploads
rag_bot_upload = Assistant(
    llm=llm_cfg,
    name='Document Assistant',
    description='Upload any document and ask questions!'
    # Note: No files parameter = WebUI shows upload button
)

print("Method 2: User Uploads")
print("- No files parameter provided")
print("- WebUI shows upload button")
print("- Users can upload PDF, DOCX, TXT, etc.")
print("- Perfect for dynamic RAG\n")

print("Usage in WebUI:")
print("1. Launch: WebUI(rag_bot_upload).run()")
print("2. User uploads document via button")
print("3. User asks questions")
print("4. Bot uses RAG automatically!")

### File Upload Flow:

1. **User uploads file** in WebUI
2. **WebUI processes file** (PDF → text, DOCX → text, etc.)
3. **RAG pipeline activates** automatically
4. **Bot answers** using document content

**Supported formats:**
- PDF (.pdf)
- Word (.docx)
- PowerPoint (.pptx)
- Text (.txt)
- HTML (.html)
- Markdown (.md)

---
## Part 6: Complete Example from Official Code

### From assistant_qwen3.py - Full Working Example

In [ ]:
# Adapted from assistant_qwen3.py (lines 114-126)

def create_production_bot():
    """From official assistant_qwen3.py"""
    bot = Assistant(
        llm=llm_cfg,
        function_list=['code_interpreter'],  # Tools available
        name='Qwen3 Tool-calling Demo',
        description="I'm a demo using Qwen3 tool calling. Welcome to add and play with your own tools!"
    )
    return bot

def app_gui():
    """Launch WebUI (from official code)"""
    bot = create_production_bot()
    
    chatbot_config = {
        'prompt.suggestions': [
            'What time is it?',
            'https://github.com/orgs/QwenLM/repositories Extract markdown content of this page, then draw a bar chart to display the number of stars.'
        ]
    }
    
    # Launch WebUI
    WebUI(bot, chatbot_config=chatbot_config).run()

print("✅ Production bot setup (from official code)")
print("\nTo launch: app_gui()")
print("\nFeatures:")
print("- Tool calling (code_interpreter)")
print("- Suggested prompts")
print("- File uploads")
print("- Production-ready!")

---
## Part 7: Advanced Gradio Patterns

### From group_chat_demo.py - Custom UI

For maximum control, you can use Gradio directly with Qwen-Agent:

In [ ]:
# Patterns from group_chat_demo.py

# 1. File Upload Handling (lines 260-263)
def add_file(file):
    """Handle file uploads"""
    uploaded_file = file.name
    is_first_upload = True
    return file.name

# 2. Add User Message (lines 238-246)
def add_text(text, user_name='user'):
    """Add user message to conversation"""
    from qwen_agent.llm.schema import ContentItem, Message
    
    content = [ContentItem(text=text)]
    # Can also add file: ContentItem(file=uploaded_file)
    
    message = Message('user', content=content, name=user_name)
    return message

# 3. Clear Chat (lines 250-252)
def chat_clear():
    """Clear conversation history"""
    messages = []
    return None

print("✅ Advanced patterns from official group_chat_demo.py:")
print("\n1. File upload handling")
print("2. Message management with ContentItem")
print("3. Chat history clearing")
print("\nThese patterns give you full control over the UI!")

### Gradio UI Structure from group_chat_demo.py

```python
# From lines 271-298
import gradio as gr

with gr.Blocks(theme='soft') as demo:
    with gr.Tab('Chat'):
        with gr.Column():
            chatbot = gr.Chatbot(height=750, show_copy_button=True)
            
            with gr.Row():
                chat_txt = gr.Textbox(
                    placeholder='Chat with Qwen...',
                    container=False
                )
                chat_btn = gr.Button('Send')
                clear_btn = gr.Button('Clear')
            
            # Wire up events
            chat_txt.submit(add_text, [chat_txt], [chatbot, chat_txt])
            clear_btn.click(chat_clear, None, [chatbot])

demo.launch()
```

**Use custom Gradio when you need:**
- Multiple tabs
- Custom layouts
- Additional input fields
- Integration with other components

---
## Part 8: Three Interface Modes

### From assistant_qwen3.py - Complete Interface Options

In [ ]:
# From assistant_qwen3.py showing three interface modes

def test(query='What is 2+2?'):
    """Mode 1: Single query (testing)"""
    bot = Assistant(llm=llm_cfg)
    messages = [{'role': 'user', 'content': query}]
    
    for response in bot.run(messages=messages):
        print(response[-1].get('content', ''))

def app_tui():
    """Mode 2: Terminal UI (interactive CLI)"""
    bot = Assistant(llm=llm_cfg)
    messages = []
    
    while True:
        query = input('user question: ')
        if query.lower() in ['exit', 'quit']:
            break
        
        messages.append({'role': 'user', 'content': query})
        response = []
        for response in bot.run(messages=messages):
            print(response[-1].get('content', ''))
        messages.extend(response)

def app_gui_simple():
    """Mode 3: Web UI (for everyone)"""
    bot = Assistant(llm=llm_cfg)
    WebUI(bot).run()

print("Three Interface Modes (from official code):\n")
print("1. test() - Single query testing")
print("2. app_tui() - Terminal interactive chat")
print("3. app_gui() - Web browser interface\n")

# Demo mode 1
print("Demo: Mode 1 (Single Query)")
test('What is 15 factorial?')

---
## Part 9: Deployment Options

### How to Deploy Your WebUI

In [ ]:
# Deployment examples

print("Deployment Options:\n")

print("1. LOCAL DEVELOPMENT")
print("   WebUI(bot).run()")
print("   - Opens localhost:7860")
print("   - For testing and development\n")

print("2. SHARE LINK (Gradio)")
print("   WebUI(bot).run(share=True)")
print("   - Creates public URL (72hr limit)")
print("   - Perfect for demos")
print("   - Example: https://xxxxx.gradio.live\n")

print("3. CUSTOM HOST/PORT")
print("   WebUI(bot).run(server_name='0.0.0.0', server_port=8080)")
print("   - Accessible on local network")
print("   - Specify custom port\n")

print("4. PRODUCTION DEPLOYMENT")
print("   a) Docker:")
print("      - Use Gradio Docker images")
print("      - Scale with load balancer")
print("   b) Cloud:")
print("      - Hugging Face Spaces (free!)")
print("      - AWS, GCP, Azure")
print("   c) Add authentication:")
print("      - WebUI(bot).run(auth=('user', 'pass'))")
print("   d) HTTPS:")
print("      - Use nginx reverse proxy")
print("      - Let's Encrypt SSL certificates")

### Production Checklist:

✅ **Security:**
- Add authentication (`auth=('user', 'pass')`)
- Use HTTPS (reverse proxy)
- Validate uploaded files
- Set file size limits

✅ **Performance:**
- Use proper LLM hosting (not local)
- Enable streaming for better UX
- Cache responses when possible
- Monitor API quotas

✅ **Reliability:**
- Error handling for failed API calls
- Timeout settings
- Logging and monitoring
- Backup conversation history

✅ **User Experience:**
- Clear error messages
- Loading indicators
- Suggested prompts
- Help documentation

---
## Part 10: Complete Production Example

In [ ]:
# Complete production-ready example

def create_production_assistant():
    """Create a production-ready assistant"""
    
    bot = Assistant(
        llm=llm_cfg,
        function_list=['code_interpreter'],
        name='Production Assistant',
        description='A production-ready AI assistant with tools and RAG',
        system_message='''You are a helpful AI assistant.
        
Guidelines:
- Be concise and clear
- Use tools when appropriate
- Cite sources when using documents
- Admit when you don't know something'''
    )
    
    return bot

def launch_production_ui():
    """Launch production WebUI with all features"""
    
    bot = create_production_assistant()
    
    chatbot_config = {
        'prompt.suggestions': [
            'What can you help me with?',
            'Calculate the factorial of 20',
            'Analyze this document (upload first)',
            'Create a Python script to sort a list'
        ]
    }
    
    # Launch with production settings
    WebUI(
        bot,
        chatbot_config=chatbot_config
    ).run(
        # Production settings:
        server_name='0.0.0.0',  # Accept external connections
        server_port=7860,        # Standard port
        share=False,             # No public link in production
        # auth=('admin', 'password123'),  # Uncomment to add auth
    )

print("✅ Production setup complete!\n")
print("Features included:")
print("- Tool calling (code_interpreter)")
print("- File uploads for RAG")
print("- Suggested prompts")
print("- System message guidelines")
print("- Production deployment settings\n")

print("To launch: launch_production_ui()")
print("\nFor Jupyter notebooks, we don't launch the UI")
print("(would block execution)")

---
## Part 11: Comparison - CLI vs WebUI

In [ ]:
# Same bot, two interfaces

bot = Assistant(
    llm=llm_cfg,
    function_list=['code_interpreter']
)

print("INTERFACE COMPARISON\n")
print("="*60)

print("\n1. CLI Interface (for developers):")
print("-" * 60)
messages = [{'role': 'user', 'content': 'What is 42 * 137?'}]
for response in bot.run(messages):
    if response:
        print(f"Answer: {response[-1].get('content', '')}")
        break

print("\nPros: Fast, scriptable, no dependencies")
print("Cons: Developer-only, no history, no files\n")

print("="*60)
print("\n2. WebUI (for everyone):")
print("-" * 60)
print("WebUI(bot).run()\n")
print("Features:")
print("✅ Beautiful chat interface")
print("✅ Message history")
print("✅ File uploads")
print("✅ Copy buttons")
print("✅ No coding required")
print("✅ Accessible to anyone\n")

print("Pros: User-friendly, full features, production-ready")
print("Cons: Requires server, slightly more complex deployment")
print("\n" + "="*60)

---
## Summary

✅ **WebUI basics** - One line: `WebUI(bot).run()`

✅ **chatbot_config** - Customize with prompt.suggestions

✅ **Tools in WebUI** - code_interpreter, MCP, custom tools all work

✅ **File uploads** - Automatic RAG in the browser

✅ **Advanced Gradio** - Custom UI from group_chat_demo.py

✅ **Three modes** - test(), app_tui(), app_gui()

✅ **Deployment** - Local, share, Docker, production

✅ **All from official code** - assistant_qwen3.py, group_chat_demo.py

### Key Takeaways:

1. **WebUI makes agents accessible** - From CLI to everyone
2. **One line to web interface** - `WebUI(bot).run()`
3. **File uploads = automatic RAG** - No extra code needed
4. **Tools work seamlessly** - Same bot, better interface
5. **Production-ready** - Auth, HTTPS, scaling all supported

### From Official Examples:
- `assistant_qwen3.py` - WebUI with tools and MCP
- `group_chat_demo.py` - Advanced Gradio patterns
- All configurations tested in production!

---

## 🎉 Course Complete!

### Your 12-Day Journey:

**Foundation (Days 1-3):**
- LLM basics and API setup
- Messages and conversations
- Streaming and error handling

**Building Blocks (Days 4-7):**
- Built-in tools (CodeInterpreter, DocParser)
- Your first agent
- Function calling patterns
- Custom tool development

**Advanced Features (Days 8-10):**
- Assistant agent mastery
- RAG systems and document intelligence
- Multi-agent coordination

**Production Ready (Days 11-12):**
- Reasoning models (QwQ, Qwen3 thinking)
- Web interfaces (WebUI, Gradio)
- Deployment and production patterns

### You Can Now Build:
- 🤖 Production AI agents with tools
- 📚 Document Q&A systems with RAG
- 🤝 Multi-agent teams
- 🧠 Reasoning assistants
- 🎨 Web interfaces for everyone

### Next Steps:
1. **Build your first app** - Start with a simple use case
2. **Explore official examples** - `/examples` directory
3. **Read the docs** - [Qwen-Agent Documentation](https://github.com/QwenLM/Qwen-Agent)
4. **Join the community** - Share your creations!

**You're ready to build production AI agents! 🚀**